In [ ]:
from brian2 import *
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from src.nb_helpers import *

%matplotlib inline
seed(12345)

In [ ]:
RECALCULATE_EQUILIBRIUM = False
PLOT_ONLY_FROM_EQUILIBRIUM=True
BIN_SIZE_FIRING_RATE = BIN_SIZE_ISI = 10
USE_SYNAPSE_PROBS = True
USE_DENDRITE_MODEL = True

index_to_ntype_dict = {
    0: 'CS',
    1: 'CC',
    2: 'SST',
    3: 'PV'
}

equilibrium_t = 0.2*second

In [ ]:
start_scope()

################################################################################
# Model parameters
################################################################################
### General parameters
duration = 3*second  # Total simulation time
sim_dt = 0.1*ms        # Integrator/sampling step

# N_sst = N_pv = 100 # Number of SST neurons & PV neurons each (inhibitory)
# N_cc = N_cs = 400 # Number of CC neurons & CS neurons each (excitatory)
N_sst = 10 # Number of SST neurons (inhibitory)
N_pv = 10  # Number of PV neurons (inhibitory)
N_cc = 40  # Number of CC (PYR) neurons (excitatory)
N_cs = 40  # Number of CS (PYR) neurons (excitatory)

### Neuron parameters
tau_S   = 16*ms  # PYR neuron - soma membrane time constant 
tau_D   =  7*ms  # PYR neuron - dendritic membrane time constant
tau_SST = 20*ms  # SST neuron membrane time constant
tau_PV  = 10*ms  # PV neuron membrane time constant
tau_E   =  5*ms  # Excitatory synaptic time constant
tau_I   = 10*ms  # Inhibitory synaptic time constant

C_S   = 370*pF  # PYR neuron - soma membrane capacitance
C_D   = 170*pF  # PYR neuron - dendritic membrane capacitance
C_SST = 100*pF  # SST neuron membrane capacitance
C_PV  = 100*pF  # PV neuron membrane capacitance
 
E_l  = -70*mV   # leak reversal potential
E_e  =   0*mV   # Excitatory synaptic reversal potential
E_i  = -80*mV   # Inhibitory synaptic reversal potential

V_t  = -50*mV   # spiking threashold
V_r  = E_l      # reset potential

c_d = 2600 * pA  # back-propagates somatic spikes to to the dendrites
g_s = 1300 * pA  # propagates dendritic regenerative activity to soma
g_d = 1200 * pA  # propagates dendritic regenerative activity to denderites

### Connectivity weight & probabilities
# CS_CS 0, CS_SST 1, CS_PV 2, SST_CS 3, PV_CS 4, CC_CC 5, CC_SST 6, CC_PV 7, SST_CC 8, PV_CC 9, CC_CS 10, SST_PV 11, SST_SST 12, PV_PV 13, PV_SST 14, 
conn_weights = [0.27, 0.05, 1.01, 0.19, 0.32, 0.24, 0.09, 0.48, 0.19, 0.52, 0.19, 0.18, 0.19, 0.47, 0.44] # in nS
if USE_SYNAPSE_PROBS: 
    # CS_CS 0, CS_SST 1, CS_PV 2, SST_CS 3, PV_CS 4, CC_CC 5, CC_SST 6, CC_PV 7, SST_CC 8, PV_CC 9, CC_CS 10, SST_PV 11, SST_SST 12, PV_PV 13, PV_SST 14, 
    conn_probs = [0.16, 0.23, 0.18, 0.52, 0.43, 0.06, 0.26, 0.22, 0.13, 0.38, 0.09, 0.29, 0.1, 0.5, 0.14]
# If not `USE_SYNAPSE_PROBS`, the topology is maintained as specified with a probability p=1
else: 
    conn_probs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### External Input
g = 50
M = 1.05 
I_ext_sst = [g for i in range(N_sst)]
I_ext_pv = [g for i in range(N_pv)]
I_ext_cc = [g for i in range(N_cc)]
I_ext_cs = [g for i in range(N_cs)]
lambda_cs = lambda_cc = lambda_sst = lambda_pv = 10*Hz

################################################################################

In [ ]:
### Sigmoid function params
E_d = -38*mV  # position control of threshold
D_d =   6*mV  # sharpness control of threshold 

# TODO see how to reference this from equation 
@check_units(x=volt, result=1)
def sigmoid(x):
    return 1/(1+np.exp(-(-x-E_d)/D_d))

In [ ]:
# Equations for SST (inhibitory) neurons
eqs_sst_inh = '''
    dv/dt = ((E_l-v)/tau_SST + I/C_SST) : volt (unless refractory)

    dg_e/dt = -g_e/tau_E : siemens
    dg_i/dt = -g_i/tau_I : siemens
    
    I = g_e*(E_e - v) + g_i*(E_i - v) : amp
'''

# Equations for PV (inhibitory) neurons
eqs_pv_inh = '''
    dv/dt = ((E_l-v)/tau_PV + I/C_PV) : volt (unless refractory)

    dg_e/dt = -g_e/tau_E : siemens
    dg_i/dt = -g_i/tau_I : siemens

    I = g_e*(E_e - v) + g_i*(E_i - v) : amp
'''

# Equations for PYR (excitatory) neurons WITH dendrites
eqs_exc_with_dendrite = '''
    dv_s/dt = ((E_l-v_s)/tau_S + (g_s*(1/(1+exp(-(v_d-E_d)/D_d))) + I_s)/C_S) : volt (unless refractory)

    dg_es/dt = -g_es/tau_E : siemens
    dg_is/dt = -g_is/tau_I : siemens

    I_s = g_es*(E_e - v_s) + g_is*(E_i - v_s) : amp

    dv_d/dt = ((E_l-v_d)/tau_D + (g_d*(1/(1+exp(-(v_d-E_d)/D_d))) + c_d*K + I_d)/C_D) : volt

    dg_ed/dt = -g_ed/tau_E : siemens
    dg_id/dt = -g_id/tau_I : siemens

    I_d = g_ed*(E_e - v_d) + g_id*(E_i - v_d) : amp
    K : 1
'''

# Equations for PYR (excitatory) neurons WITHOUT dendrites
eqs_exc_without_dendrite = '''
    dv_s/dt = ((E_l-v_s)/tau_S + I_s/C_S) : volt (unless refractory)

    dg_es/dt = -g_es/tau_E : siemens
    dg_is/dt = -g_is/tau_I : siemens

    I_s = g_es*(E_e - v_s) + g_is*(E_i - v_s) : amp
    K : 1
'''

In [ ]:
# SST Neurons
sst_neurons = NeuronGroup(N_sst, model=eqs_sst_inh, threshold='v > V_t',
                              reset='v = E_l', refractory=8.3 * ms, method='euler')
sst_neurons.v = 'E_l + rand()*(V_t-E_l)'

## Poisson input to SST neurons
for n_idx in range(N_sst):
    sst_input_i = PoissonInput(sst_neurons, 'g_e', N=1, rate=lambda_sst, weight=I_ext_sst[n_idx]*nS)

# PV Neurons
pv_neurons = NeuronGroup(N_pv, model=eqs_pv_inh, threshold='v > V_t',
                             reset='v = E_l', refractory=8.3 * ms, method='euler')
pv_neurons.v = 'E_l + rand()*(V_t-E_l)'

## Poisson input to PV neurons
for n_idx in range(N_pv):
    pv_input_i = PoissonInput(pv_neurons, 'g_e', N=1, rate=lambda_pv, weight=I_ext_pv[n_idx]*nS)

# CS Neurons
if USE_DENDRITE_MODEL:
    cs_neurons = NeuronGroup(N_cs, model=eqs_exc_with_dendrite, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cs_neurons.v_s = 'E_l + rand()*(V_t-E_l)'
    cs_neurons.v_d = -70 * mV
else:
    cs_neurons = NeuronGroup(N_cs, model=eqs_exc_without_dendrite, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cs_neurons.v_s = 'E_l + rand()*(V_t-E_l)'

## Poisson input to CS neurons
for n_idx in range(N_cs):
    cs_input_i = PoissonInput(cs_neurons, 'g_es', N=1, rate=lambda_cs, weight=I_ext_cs[n_idx]*nS)

# CC Neurons
if USE_DENDRITE_MODEL:
    cc_neurons = NeuronGroup(N_cc, model=eqs_exc_with_dendrite, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cc_neurons.v_s = 'E_l + rand()*(V_t-E_l)'
    cc_neurons.v_d = -70 * mV
else:
    cc_neurons = NeuronGroup(N_cc, model=eqs_exc_without_dendrite, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cc_neurons.v_s = 'E_l + rand()*(V_t-E_l)'

## Poisson input to CC neurons
for n_idx in range(N_cc):
    cc_input_i = PoissonInput(cc_neurons, 'g_es', N=1, rate=lambda_cc, weight=I_ext_cc[n_idx]*nS)

In [ ]:
# CS_CS  0, CS_SST 1, CS_PV  2, SST_CS  3, PV_CS    4, CC_CC  5, CC_SST  6, CC_PV 7
# SST_CC 8, PV_CC  9, CC_CS 10, SST_PV 11, SST_SST 12, PV_PV 13, PV_SST 14

# SST <=> PV
SST_PV = 11
conn_SST_PV = Synapses(sst_neurons, pv_neurons, model='w: 1', on_pre='g_i+=w*nS', name='SST_PV') # inhibitory
conn_SST_PV.connect(p=conn_probs[SST_PV])
conn_SST_PV.w = conn_weights[SST_PV]

PV_SST = 14
conn_PV_SST = Synapses(pv_neurons, sst_neurons, model='w: 1', on_pre='g_i+=w*nS', name='PV_SST') # inhibitory
conn_PV_SST.connect(p=conn_probs[PV_SST])
conn_PV_SST.w = conn_weights[PV_SST]

# PV <=> PYR soma
## target CS soma
PV_CSsoma = 4
conn_PV_CSsoma = Synapses(pv_neurons, cs_neurons, model='w: 1', on_pre='g_is+=w*nS', name='PV_CSsoma') # inhibitory
conn_PV_CSsoma.connect(p=conn_probs[PV_CSsoma])
conn_PV_CSsoma.w = conn_weights[PV_CSsoma]

CSsoma_PV = 2
conn_CSsoma_PV = Synapses(cs_neurons, pv_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CSsoma_PV') # excitatory
conn_CSsoma_PV.connect(p=conn_probs[CSsoma_PV])
conn_CSsoma_PV.w = conn_weights[CSsoma_PV]

## target CC soma
PV_CCsoma = 9
conn_PV_CCsoma = Synapses(pv_neurons, cc_neurons, model='w: 1', on_pre='g_is+=w*nS', name='PV_CCsoma') # inhibitory 
conn_PV_CCsoma.connect(p=conn_probs[PV_CCsoma])
conn_PV_CCsoma.w = conn_weights[PV_CCsoma]


CCsoma_PV = 7
conn_CCsoma_PV = Synapses(cc_neurons, pv_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CCsoma_PV') # excitatory
conn_CCsoma_PV.connect(p=conn_probs[CCsoma_PV])
conn_CCsoma_PV.w = conn_weights[CCsoma_PV]

# SST <=> PYR soma
## target CS soma
SST_CSsoma = 3
conn_SST_CSsoma = Synapses(sst_neurons, cs_neurons, model='w: 1', on_pre='g_is+=w*nS', name='SST_CSsoma') # inhibitory (optional connection)
conn_SST_CSsoma_prob = conn_probs[SST_CSsoma] / 2 if USE_DENDRITE_MODEL else conn_probs[SST_CSsoma] # prob divided between soma & dendrite only if `USE_DENDRITE_MODEL`
conn_SST_CSsoma.connect(p=conn_SST_CSsoma_prob) 
conn_SST_CSsoma.w = conn_weights[SST_CSsoma]

CSsoma_SST = 1
conn_CSsoma_SST = Synapses(cs_neurons, sst_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CSsoma_SST') # excitatory
conn_CSsoma_SST.connect(p=conn_probs[CSsoma_SST])
conn_CSsoma_SST.w = conn_weights[CSsoma_SST]

## taget CC soma
SST_CCsoma = 8
conn_SST_CCsoma = Synapses(sst_neurons, cc_neurons, model='w: 1', on_pre='g_is+=w*nS', name='SST_CCsoma') # inhibitory (optional connection)
conn_SST_CSsoma_prob = conn_probs[SST_CCsoma] / 2 if USE_DENDRITE_MODEL else conn_probs[SST_CCsoma] # prob divided between soma & dendrite only if `USE_DENDRITE_MODEL`
conn_SST_CCsoma.connect(p=conn_SST_CSsoma_prob)
conn_SST_CCsoma.w = conn_weights[SST_CCsoma]

CCsoma_SST = 6
conn_CCsoma_SST = Synapses(cc_neurons, sst_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CCsoma_SST') # excitatory
conn_CCsoma_SST.connect(p=conn_probs[CCsoma_SST])
conn_CCsoma_SST.w = conn_weights[CCsoma_SST]

# CC => CS 
## target CS soma
CCsoma_CSsoma = 10
conn_CCsoma_CSsoma = Synapses(cc_neurons, cs_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CC_CSsoma') # excitatory
conn_CCsoma_CSsoma.connect(p=conn_probs[CCsoma_CSsoma])
conn_CCsoma_CSsoma.w = conn_weights[CCsoma_CSsoma]

# self connections
## CS soma self connection
CSsoma_CSsoma = 0
conn_CSsoma_CSsoma = Synapses(cs_neurons, cs_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CSsoma_CSsoma')  # excitatory
conn_CSsoma_CSsoma.connect(p=conn_probs[CSsoma_CSsoma])
conn_CSsoma_CSsoma.w = conn_weights[CSsoma_CSsoma]

backprop_CS = Synapses(cs_neurons, cs_neurons, on_pre={'up': 'K += 1', 'down': 'K -=1'},
                           delay={'up': 0.5 * ms, 'down': 2 * ms}, name='backprop_CS')
backprop_CS.connect(condition='i==j')  # Connect all CS neurons to themselves

## CC soma self connection
CCsoma_CCsoma = 5
conn_CCsoma_CCsoma = Synapses(cc_neurons, cc_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CCsoma_CCsoma')  # excitatory
conn_CCsoma_CCsoma.connect(p=conn_probs[CCsoma_CCsoma])
conn_CCsoma_CCsoma.w = conn_weights[CCsoma_CCsoma]

backprop_CC = Synapses(cc_neurons, cc_neurons, on_pre={'up': 'K += 1', 'down': 'K -=1'},
                           delay={'up': 0.5 * ms, 'down': 2 * ms}, name='backprop_CC')
backprop_CC.connect(condition='i==j')  # Connect all CC neurons to themselves

## SST self connection
SST_SST = 12
conn_SST_SST = Synapses(sst_neurons, sst_neurons, model='w: 1', on_pre='g_i+=w*nS', name='SST_SST')  # inhibitory
conn_SST_SST.connect(p=conn_probs[SST_SST])
conn_SST_SST.w = conn_weights[SST_SST]

## PV self connection
PV_PV = 13
conn_PV_PV = Synapses(pv_neurons, pv_neurons, model='w: 1', on_pre='g_i+=w*nS', name='PV_PV')  # inhibitory
conn_PV_PV.connect(p=conn_probs[PV_PV])
conn_PV_PV.w = conn_weights[PV_PV]


# SST => PYR dendrite
if USE_DENDRITE_MODEL:
    ## target CS dendrite
    SST_CSdendrite = 3
    conn_SST_CSdendrite = Synapses(sst_neurons, cs_neurons, model='w: 1', on_pre='g_id+=w*nS', name='SST_CSdendrite') # inhibitory
    conn_SST_CSdendrite.connect(p=conn_probs[SST_CSdendrite] / 2) # prob divided between soma & dendrite
    conn_SST_CSdendrite.w = conn_weights[SST_CSdendrite]

    ## target CC dendrite
    SST_CCdendrite = 8
    conn_SST_CCdendrite = Synapses(sst_neurons, cc_neurons, model='w: 1', on_pre='g_id+=w*nS', name='SST_CCdendrite') # inhibitory
    conn_SST_CCdendrite.connect(p=conn_probs[SST_CCdendrite] / 2) # prob divided between soma & dendrite
    conn_SST_CCdendrite.w = conn_weights[SST_CCdendrite]



In [ ]:
# ##############################################################################
# # Monitors
# ##############################################################################

# Record spikes of different neuron groups
spike_mon_sst = SpikeMonitor(sst_neurons)
spike_mon_pv = SpikeMonitor(pv_neurons)
spike_mon_cs = SpikeMonitor(cs_neurons)
spike_mon_cc = SpikeMonitor(cc_neurons)

# Record conductances and membrane potential of neuron ni
inh_neuron_variables = ['v', 'g_e', 'g_i']
state_mon_sst = StateMonitor(sst_neurons, inh_neuron_variables, record=[0])
state_mon_pv = StateMonitor(pv_neurons, inh_neuron_variables, record=[0])

exc_neuron_variables = ['v_s', 'v_d', 'g_es', 'g_is', 'g_ed', 'g_id'] if USE_DENDRITE_MODEL else ['v_s', 'g_es', 'g_is']
state_mon_cs = StateMonitor(cs_neurons, exc_neuron_variables, record=[0])
state_mon_cc = StateMonitor(cc_neurons, exc_neuron_variables, record=[0])

In [ ]:
# ##############################################################################
# # Simulation run
# ##############################################################################

defaultclock.dt = sim_dt

run(duration, report='text')

In [ ]:
TIME_FRAME = 0.1

if RECALCULATE_EQUILIBRIUM:
    equilibrium_times = []
    for idx, spike_mon in enumerate([spike_mon_cs, spike_mon_cc, spike_mon_sst, spike_mon_pv ]):
        t, firing_rate = compute_equilibrium_for_neuron_type(spike_mon, TIME_FRAME)
        if firing_rate is not None:
            print(f"Found for {index_to_ntype_dict[idx]} neurons")

        equilibrium_times.append(t)

    equilibrium_t = max(equilibrium_times)*second
    if equilibrium_t < duration:
        print(f"Equilibrium for all neurons start at: {equilibrium_t}")
    else: 
        print(f"WARNING: Equilibrium was not found during the duration of the simulation")
else:
    print(f"Skipping recalculating equilibrium time. Using eq_t={equilibrium_t}")

In [ ]:
################################################################################
# Analysis and plotting
################################################################################

# Raster plot
N = N_cs + N_cc + N_sst + N_pv
eventplot(spike_mon_cs.i[:,np.newaxis], lineoffsets=spike_mon_cs.t / ms, orientation='vertical', colors='b', linewidths=5)
# axhline(N_cs - 1/2, lw=0.5, color='k')
eventplot((N_cs + spike_mon_cc.i)[:,np.newaxis], lineoffsets=spike_mon_cc.t / ms, orientation='vertical', colors='r', linewidths=5)
# axhline(N_cs + N_cc - 1/2, lw=0.5, color='k')
eventplot(((N_cs + N_cc) + spike_mon_sst.i)[:,np.newaxis], lineoffsets=spike_mon_sst.t / ms, orientation='vertical', colors='g', linewidths=5)
# axhline(N_cs + N_cc + N_sst - 1/2, lw=0.5, color='k')
eventplot(((N_cs + N_cc + N_sst) + spike_mon_pv.i)[:,np.newaxis], lineoffsets=spike_mon_pv.t / ms, orientation='vertical', colors='y', linewidths=5)
ylim(bottom=-1/2, top=N - 1/2)
xlabel('Time (ms)')
ylabel('Neuron index')

custom_handles = [Line2D([0], [0], color='y', lw=2, label='PV'),
                  Line2D([0], [0], color='g', lw=2, label='SST'),
                  Line2D([0], [0], color='r', lw=2, label='CS'),
                  Line2D([0], [0], color='b', lw=2, label='CC')]
legend(handles=custom_handles, loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

In [ ]:
# CS Neurons plot

figure(figsize=(18, 4))
subplot(1, 2, 1)
plot(state_mon_cs.t / ms, state_mon_cs.v_s[0], label='v_s')
if USE_DENDRITE_MODEL:
    plot(state_mon_cs.t / ms, state_mon_cs.v_d[0], label='v_d')
for (t, i) in zip(spike_mon_cs.t, spike_mon_cs.i):
    if i == 0:
        axvline(t / ms, ls='--', c='C1', lw=1)
axhline(V_t / mV / 1000, ls=':', c='C2', lw=3, label='spike thld')
xlabel('Time (ms)')
ylabel('potential (V)')
legend(loc='upper right')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

subplot(1, 2, 2)
plot(state_mon_cs.t / ms, state_mon_cs.g_es[0], label='g_es')
plot(state_mon_cs.t / ms, state_mon_cs.g_is[0], label='g_is')
if USE_DENDRITE_MODEL:
    plot(state_mon_cs.t / ms, state_mon_cs.g_ed[0], label='g_ed')
    plot(state_mon_cs.t / ms, state_mon_cs.g_id[0], label='g_id')

xlabel('Time (ms)')
ylabel('Conductance (S)')
legend(loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

In [ ]:
# CC Neurons

figure(figsize=(18, 4))
subplot(1, 2, 1)
plot(state_mon_cc.t / ms, state_mon_cc.v_s[0], label='v_s')
if USE_DENDRITE_MODEL:
    plot(state_mon_cc.t / ms, state_mon_cc.v_d[0], label='v_d')
for (t, i) in zip(spike_mon_cc.t, spike_mon_cc.i):
    if i == 0:
        axvline(t / ms, ls='--', c='C1', lw=1)
axhline(V_t / mV / 1000, ls=':', c='C2', lw=3, label='spike thld')
xlabel('Time (ms)')
ylabel('membrane potential (V)')
legend(loc='upper right')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

subplot(1, 2, 2)
plot(state_mon_cc.t / ms, state_mon_cc.g_es[0], label='g_es')
plot(state_mon_cc.t / ms, state_mon_cc.g_is[0], label='g_is')
if USE_DENDRITE_MODEL:
    plot(state_mon_cc.t / ms, state_mon_cc.g_ed[0], label='g_ed')
    plot(state_mon_cc.t / ms, state_mon_cc.g_id[0], label='g_id')

xlabel('Time (ms)')
ylabel('Conductance (S)')
legend(loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

In [ ]:
# SST Neurons plot

figure(figsize=(18, 4))
subplot(1, 2, 1)
plot(state_mon_sst.t / ms, state_mon_sst.v[0], label='v')
for (t, i) in zip(spike_mon_sst.t, spike_mon_sst.i):
    if i == 0:
        axvline(t / ms, ls='--', c='C1', lw=1)
axhline(V_t / mV / 1000, ls=':', c='C2', lw=3, label='spike thld')
xlabel('Time (ms)')
ylabel('membrane potential (V)')
legend(loc='upper right')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

subplot(1, 2, 2)
plot(state_mon_sst.t / ms, state_mon_sst.g_e[0], label='g_e')
plot(state_mon_sst.t / ms, state_mon_sst.g_i[0], label='g_i')

xlabel('Time (ms)')
ylabel('Conductance (S)')
legend(loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

In [ ]:
# PV Neurons plot

figure(figsize=(18, 4))
subplot(1, 2, 1)
plot(state_mon_pv.t / ms, state_mon_pv.v[0], label='v')
for (t, i) in zip(spike_mon_pv.t, spike_mon_pv.i):
    if i == 0:
        axvline(t / ms, ls='--', c='C1', lw=1)
axhline(V_t / mV / 1000, ls=':', c='C2', lw=3, label='spike thld')
xlabel('Time (ms)')
ylabel('membrane potential (V)')
legend(loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

subplot(1, 2, 2)
plot(state_mon_pv.t / ms, state_mon_pv.g_e[0], label='g_e')
plot(state_mon_pv.t / ms, state_mon_pv.g_i[0], label='g_i')

xlabel('Time (ms)')
ylabel('Conductance (S)')
legend(loc='best')

if PLOT_ONLY_FROM_EQUILIBRIUM:
    xlim(left=equilibrium_t/ms, right=duration/ms)

In [ ]:
from_t = equilibrium_t if PLOT_ONLY_FROM_EQUILIBRIUM else 0
to_t = duration

# Compute firing rate for each neuron group
firing_rates_cs = compute_firing_rate_for_neuron_type(spike_mon_cs, from_t, to_t)
firing_rates_cc = compute_firing_rate_for_neuron_type(spike_mon_cc, from_t, to_t)
firing_rates_sst = compute_firing_rate_for_neuron_type(spike_mon_sst, from_t, to_t)
firing_rates_pv = compute_firing_rate_for_neuron_type(spike_mon_pv, from_t, to_t)

print(f'Avg firing rate for CS neurons: {np.mean(firing_rates_cs) * Hz}')
print(f'Avg firing rate for CC neurons: {np.mean(firing_rates_cc) * Hz}')
print(f'Avg firing rate for SST neurons: {np.mean(firing_rates_sst) * Hz}')
print(f'Avg firing rate for PV neurons: {np.mean(firing_rates_pv) * Hz}')

In [ ]:
# Compute input & output selectivity for CC & CS neuron groups
input_selectivity = compute_input_selectivity(I_ext_cs)
print(f'Input selectivity: {input_selectivity}')

output_selectivity_cs = compute_output_selectivity_for_neuron_type(spike_mon_cs, from_t, to_t)
output_selectivity_cc = compute_output_selectivity_for_neuron_type(spike_mon_cc, from_t, to_t)

print(f'Output selectivity CS: {output_selectivity_cs}')
print(f'Output selectivity CC: {output_selectivity_cc}')

In [ ]:
# Plot histogram of firing rates

firing_rates = [firing_rates_cs, firing_rates_cc, firing_rates_sst, firing_rates_pv]

columns = 2
rows = int(len(firing_rates) / columns)

fig, axs = plt.subplots(rows, columns, figsize = (12, 9))

for (ntype_index, firing_rate_i) in enumerate(firing_rates):
    row_idx = int(ntype_index / columns)
    col_idx = ntype_index % columns
               
    axs[row_idx][col_idx].hist(firing_rate_i, bins=BIN_SIZE_FIRING_RATE)
    axs[row_idx][col_idx].axis(ymin=0)
    axs[row_idx][col_idx].set_title(f'Neuron group {index_to_ntype_dict[ntype_index]}', fontsize = 10)
    axs[row_idx][col_idx].set_ylabel("Frequency", fontsize = 10)
    axs[row_idx][col_idx].set_xlabel("Firing rate [Hz]", fontsize = 10)
    axs[row_idx][col_idx].tick_params(axis='both', which='major', labelsize=10)

    
plt.tight_layout()
plt.show()

In [ ]:
# Plot histogram of interspike intervals

interspike_intervals_cs = np.concatenate(compute_interspike_intervals(spike_mon_cs, from_t, to_t), axis=0)
interspike_intervals_cc = np.concatenate(compute_interspike_intervals(spike_mon_cc, from_t, to_t), axis=0)
interspike_intervals_sst = np.concatenate(compute_interspike_intervals(spike_mon_sst, from_t, to_t), axis=0)
interspike_intervals_pv = np.concatenate(compute_interspike_intervals(spike_mon_pv, from_t, to_t), axis=0)

interspike_intervals = [interspike_intervals_cs, interspike_intervals_cc, interspike_intervals_sst, interspike_intervals_pv]

columns = 2
rows = int(len(interspike_intervals) / columns)

fig, axs = plt.subplots(rows, columns, figsize = (12, 9))

for (ntype_index, interspike_intervals_i) in enumerate(interspike_intervals):
    row_idx = int(ntype_index / columns)
    col_idx = ntype_index % columns
               
    axs[row_idx][col_idx].hist(interspike_intervals_i, bins=BIN_SIZE_ISI)
    axs[row_idx][col_idx].axis(ymin=0)
    axs[row_idx][col_idx].set_title(f'Neuron group {index_to_ntype_dict[ntype_index]}', fontsize = 10)
    axs[row_idx][col_idx].set_ylabel("Frequency", fontsize = 10)
    axs[row_idx][col_idx].set_xlabel("ISI [s]", fontsize = 10)
    axs[row_idx][col_idx].tick_params(axis='both', which='major', labelsize=10)

    
plt.tight_layout()
plt.show()

In [ ]:
def plot_hist_autocorr(isi, index):
    if len(isi) == 0:
        return None
    
    sorted_isi = np.sort(isi)
    bin_dt = (sorted_isi[-1]-sorted_isi[0]) / BIN_SIZE_ISI # compute bin size based on histogram's bin size
    binned_isi = bin(sorted_isi, bin_dt)
    xaxis, acorr = compute_autocorr(binned_isi)
    xaxis = xaxis*bin_dt
    minimum = find_minimum_autocorr(acorr)

    label_minimum = f"maxISI {str(np.round(xaxis[minimum], 4))} s"

    figure(figsize=(18, 4))
    subplot(1, 2, 1)
    n, bins, patches = hist(isi, bins=BIN_SIZE_ISI)
    title(f'Neuron group {index_to_ntype_dict[index]}', fontsize = 10)
    ylabel("Frequency", fontsize = 10)
    xlabel("ISI [s]", fontsize = 10)
    tick_params(axis='both', which='major', labelsize=10)
    if minimum:
        vlines(xaxis[minimum], 0, np.max(n), label=label_minimum, color='red')
        legend()

    subplot(1, 2, 2)
    plot(xaxis, acorr, c='k')
    title(f'Neuron group {index_to_ntype_dict[index]}', fontsize = 10)
    xlabel("time lag [s]")
    ylabel("norm. autocorr.")
    if minimum:
        vlines(xaxis[minimum], 0, 1, label=label_minimum, color='red')
        legend()
        
    return xaxis[minimum] if minimum else None

In [ ]:
maxISI_cs = plot_hist_autocorr(interspike_intervals_cs, 0)

In [ ]:
maxISI_cc = plot_hist_autocorr(interspike_intervals_cc, 1)

In [ ]:
maxISI_sst = plot_hist_autocorr(interspike_intervals_sst, 2)

In [ ]:
maxISI_pv = plot_hist_autocorr(interspike_intervals_pv, 3)

In [ ]:
# Detect bursts

burst_trains_cs = compute_burst_trains(spike_mon_cs, maxISI_cs*second) if maxISI_cs else {}
burst_trains_cc = compute_burst_trains(spike_mon_cc, maxISI_cc*second) if maxISI_cc else {}
burst_trains_sst = compute_burst_trains(spike_mon_sst, maxISI_sst*second) if maxISI_sst else {}
burst_trains_pv = compute_burst_trains(spike_mon_pv, maxISI_pv*second) if maxISI_pv else {}

# Compute vector of burst lengths
burst_lengths_cs = compute_burst_lengths_by_neuron_group(burst_trains_cs)
burst_lengths_cc = compute_burst_lengths_by_neuron_group(burst_trains_cc)
burst_lengths_sst = compute_burst_lengths_by_neuron_group(burst_trains_sst)
burst_lengths_pv = compute_burst_lengths_by_neuron_group(burst_trains_pv)

print(f"Burst lengths vector for CS neurons: {len(burst_lengths_cs)} (size)")
print(f"Burst lengths vector for CC neurons: {len(burst_lengths_cc)} (size)")
print(f"Burst lengths vector for SST neurons: {len(burst_lengths_sst)} (size)")
print(f"Burst lengths vector for PV neurons: {len(burst_lengths_pv)} (size)")

In [ ]:
def visualise_synapse_group_connectivity(ax, S, S_reverse):
    Ns = len(S.source)
    Nt = len(S.target)

    ax.plot(S.i, S.j, '>b', markersize=10)
    
    if S_reverse:
        ax.plot(S_reverse.j, S_reverse.i, '<g', markersize=10)

    ax.set_xlim(-1, Ns)
    ax.set_ylim(-1, Nt)
    ax.set_xlabel('Source neuron index')
    ax.set_ylabel('Target neuron index')


def visualise_SST_connectivity(axs):
    axs[0].set_title('SST <=> SST')
    visualise_synapse_group_connectivity(axs[0], conn_SST_SST, conn_SST_SST)
    
    axs[1].set_title('SST <=> PV')
    visualise_synapse_group_connectivity(axs[1], conn_SST_PV, conn_PV_SST)
    
    axs[2].set_title('SST <=> CS Soma')
    visualise_synapse_group_connectivity(axs[2], conn_SST_CSsoma, conn_CSsoma_SST)
    
    axs[3].set_title('SST => CS Dendrite')
    if USE_DENDRITE_MODEL:
        visualise_synapse_group_connectivity(axs[3], conn_SST_CSdendrite, None)
    
    axs[4].set_title('SST <=> CC Soma')
    visualise_synapse_group_connectivity(axs[4], conn_SST_CCsoma, conn_CCsoma_SST)
    
    axs[5].set_title('SST => CC Dendrite')
    if USE_DENDRITE_MODEL:
        visualise_synapse_group_connectivity(axs[5], conn_SST_CCdendrite, None)

    
def visualise_PV_connectivity(axs):
    axs[0].set_title('PV <=> SST')
    visualise_synapse_group_connectivity(axs[0], conn_PV_SST, conn_SST_PV)
    
    axs[1].set_title('PV <=> PV')
    visualise_synapse_group_connectivity(axs[1], conn_PV_PV, conn_PV_PV)
    
    axs[2].set_title('PV <=> CS Soma')
    visualise_synapse_group_connectivity(axs[2], conn_PV_CSsoma, conn_CSsoma_PV)
    
    axs[3].set_title('PV =/= CS Dendrite')
#     visualise_synapse_group_connectivity(axs[3], None, None)
    
    axs[4].set_title('PV <=> CC Soma')
    visualise_synapse_group_connectivity(axs[4], conn_PV_CCsoma, conn_CCsoma_PV)
    
    axs[5].set_title('PV =/= CC Dendrite')
#     visualise_synapse_group_connectivity(axs[5], None, None)


def visualise_CS_connectivity(axs):
    axs[0].set_title('CS Soma <=> SST')
    visualise_synapse_group_connectivity(axs[0], conn_CSsoma_SST, conn_SST_CSsoma)
    
    axs[1].set_title('CS Soma <=> PV')
    visualise_synapse_group_connectivity(axs[1], conn_CSsoma_PV, conn_PV_CSsoma)
    
    axs[2].set_title('CS Soma <=> CS Soma')
    visualise_synapse_group_connectivity(axs[2], conn_CSsoma_CSsoma, conn_CSsoma_CSsoma)
    
    axs[3].set_title('CS Soma =/= CS Dendrite')
#     visualise_synapse_group_connectivity(axs[3], None, None)
    
    axs[4].set_title('CS Soma =/= CC Soma')
#     visualise_synapse_group_connectivity(axs[4], None, None)
    
    axs[5].set_title('CS Soma =/= CC Dendrite')
#     visualise_synapse_group_connectivity(axs[5], None, None)


def visualise_CC_connectivity(axs):
    axs[0].set_title('CC Soma <=> SST')
    visualise_synapse_group_connectivity(axs[0], conn_CCsoma_SST, conn_SST_CCsoma)
    
    axs[1].set_title('CC Soma <=> PV')
    visualise_synapse_group_connectivity(axs[1], conn_CCsoma_PV, conn_PV_CCsoma)
    
    axs[2].set_title('CC Soma <=> CS Soma')
    visualise_synapse_group_connectivity(axs[2], conn_CCsoma_CCsoma, conn_CCsoma_CCsoma)
    
    axs[3].set_title('CC Soma =/= CS Dendrite')
#     visualise_synapse_group_connectivity(axs[3], None, None)
    
    axs[4].set_title('CC Soma =/= CC Soma')
#     visualise_synapse_group_connectivity(axs[4], None, None)
    
    axs[5].set_title('CC Soma =/= CC Dendrite')
#     visualise_synapse_group_connectivity(axs[5], None, None)


def visualise_connectivity():
    fig, axs = plt.subplots(4, 6, figsize=(30, 42))
    
    visualise_SST_connectivity(axs[0])
    visualise_PV_connectivity(axs[1])
    visualise_CS_connectivity(axs[2])
    visualise_CC_connectivity(axs[3])

In [ ]:
visualise_connectivity()